In [162]:
# Cell 1: Import library yang diperlukan
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

In [163]:
# Membaca data dari file CSV
data_csv = pd.read_csv(r"C:\Users\ANANG\MLOps2\Submission_2\data\heart.csv")

In [164]:
# Cell 2: Memilih data acak untuk prediksi
columns = data_csv.columns.values
rand = random.randint(0, len(data_csv) - 1)

# Mengambil fitur
features = data_csv.values[rand]

# Membuat dictionary input
inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Age': 54,
 'Sex': 'M',
 'ChestPainType': 'ASY',
 'RestingBP': 136,
 'Cholesterol': 220,
 'FastingBS': 0,
 'RestingECG': 'Normal',
 'MaxHR': 140,
 'ExerciseAngina': 'Y',
 'Oldpeak': 3.0,
 'ST_Slope': 'Flat',
 'HeartDisease': 1}

In [165]:
# Cell 3: Mendefinisikan fungsi-fungsi helper untuk membuat TFRecord
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [166]:
# Cell 4: Fungsi untuk menyiapkan JSON request
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [167]:
# Cell 5: Fungsi untuk melakukan prediksi
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://mlops-production-6fea.up.railway.app/v1/models/heart-disease-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.5:
        return "Tidak Mengalami Penyakit Jantung"
    else:
        return "Mengalami Penyakit Jantung"

In [168]:
# Cell 6: Melakukan prediksi dan menampilkan hasil
hasil_prediksi = make_predictions(inputs)
print(f'Hasil Prediksi: {hasil_prediksi}')
print('Fitur Input:', inputs)

Hasil Prediksi: Mengalami Penyakit Jantung
Fitur Input: {'Age': 54, 'Sex': 'M', 'ChestPainType': 'ASY', 'RestingBP': 136, 'Cholesterol': 220, 'FastingBS': 0, 'RestingECG': 'Normal', 'MaxHR': 140, 'ExerciseAngina': 'Y', 'Oldpeak': 3.0, 'ST_Slope': 'Flat', 'HeartDisease': 1}
